In [7]:
sc = SparkContext()

In [6]:
import pyspark 
from pyspark import SparkContext, SparkConf
from csv import reader
from glob import glob
from operator import add
from datetime import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpld3
from mpld3 import plugins
from mpld3.utils import get_id

In [49]:
complaint = sc.textFile('../data/311.csv')

In [50]:
complaints = complaint.mapPartitions(lambda x: reader(x, delimiter = ',',quotechar = '"'))

In [51]:
header = complaints.take(2)
print header

[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name', 'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip', 'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2', 'Address Type', 'City', 'Landmark', 'Facility Type', 'Status', 'Due Date', 'Resolution Action Updated Date', 'Community Board', 'Borough', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough', 'School Name', 'School Number', 'School Region', 'School Code', 'School Phone Number', 'School Address', 'School City', 'School State', 'School Zip', 'School Not Found', 'School or Citywide Complaint', 'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment', 'Garage Lot Name', 'Ferry Direction', 'Ferry Terminal Name', 'Latitude', 'Longitude', 'Location'], ['35603714', '03/01/2017 12:00:00 AM', '03/13/201

In [41]:
import numpy as np
from collections import Counter

In [7]:
index = Counter()
for i,j in enumerate(header[0]):
    index[j] = i
print index

Counter({'Location': 51, 'Longitude': 50, 'Latitude': 49, 'Ferry Terminal Name': 48, 'Ferry Direction': 47, 'Garage Lot Name': 46, 'Bridge Highway Segment': 45, 'Road Ramp': 44, 'Bridge Highway Direction': 43, 'Bridge Highway Name': 42, 'Taxi Pick Up Location': 41, 'Taxi Company Borough': 40, 'Vehicle Type': 39, 'School or Citywide Complaint': 38, 'School Not Found': 37, 'School Zip': 36, 'School State': 35, 'School City': 34, 'School Address': 33, 'School Phone Number': 32, 'School Code': 31, 'School Region': 30, 'School Number': 29, 'School Name': 28, 'Park Borough': 27, 'Park Facility Name': 26, 'Y Coordinate (State Plane)': 25, 'X Coordinate (State Plane)': 24, 'Borough': 23, 'Community Board': 22, 'Resolution Action Updated Date': 21, 'Due Date': 20, 'Status': 19, 'Facility Type': 18, 'Landmark': 17, 'City': 16, 'Address Type': 15, 'Intersection Street 2': 14, 'Intersection Street 1': 13, 'Cross Street 2': 12, 'Cross Street 1': 11, 'Street Name': 10, 'Incident Address': 9, 'Incide

In [12]:
complaints = complaint.map(lambda x: x.encode('ascii', 'ignore')) \
                      .mapPartitions(lambda x: reader(x, delimiter = ',',quotechar = '"'))


In [13]:
def check_datetime(dt):
    try:
        d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
        if d.year < 2009 or d.year > 2017:
            return False
    except ValueError:
        return False

    return True

def get_date(dt):
    d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
    return str(d.year), str(d.month), str(d.day)


def get_date2(dt):
    d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
    return str(d.date())

def solve_time(created, closed):
    created = datetime.strptime(created, "%m/%d/%Y %I:%M:%S %p")
    try:
        closed = datetime.strptime(closed, "%m/%d/%Y %I:%M:%S %p")
    except ValueError:
        closed = datetime.now()
    return (closed - created).days

def get_week(dt):
    d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
    return d.year, d.weekday(), d.hour

In [24]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x : ((get_date(x[1]), x[3],x[5]), 1)) \
          .foldByKey(0, add) \
          .take(1)

[(((2017, 3, 8), 'HPD', 'GENERAL'), 78)]

In [59]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x : ((get_date(x[1])[0:2], x[3],x[5],x[8],x[22],x[23]), 1)) \
          .foldByKey(0, add) \
          .count()

1557687

In [37]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x: ((x[3],x[5],x[8],x[22],x[23]), solve_time(x[1], x[2]))) \
          .groupByKey() \
          .mapValues(list).take(2)
          

[(('DSNY', 'Overflowing Litter Baskets', '', '17 BROOKLYN', 'BROOKLYN'), [1]),
 (('DSNY', 'Dirty Conditions', '10013', '01 MANHATTAN', 'MANHATTAN'),
  [4, 3, 5, 3, 3])]

In [45]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x : ((get_week(x[1]),x[3],x[5],x[23]), 1)) \
          .foldByKey(0, add) \
          .take(3)

[(((2017, 6, 19), 'NYPD', 'Noise - Residential', 'BROOKLYN'), 28),
 (((2017, 6, 10), 'NYPD', 'Noise - Vehicle', 'BRONX'), 2),
 (((2017, 4, 11), 'DPR', 'New Tree Request', 'QUEENS'), 2)]

In [14]:
def sum_none(x):
    s = 0
    for i in x:
        if i == None:
            s = s
        else:
            s += i
    return s
## number of complaint by agency each day in years
num_agency_day = complaints.filter(lambda x : check_datetime(x[1])) \
                           .map(lambda x : ((get_date2(x[1]),x[3]), 1)) \
                           .foldByKey(0, add) 
            
agency = complaints.filter(lambda x : check_datetime(x[1])) \
                   .map(lambda x : x[3]) \
                   .distinct()

date = complaints.filter(lambda x : check_datetime(x[1])) \
                 .map(lambda x : get_date2(x[1])) \
                 .distinct()
        
agency_day = date.cartesian(agency) \
                 .map(lambda x : (x, 0)) \
                 .leftOuterJoin(num_agency_day) \
                 .mapValues(sum_none) \
                 .map(lambda x : [x[0][0], x[0][1], x[1]]) \
                 .collect()

In [61]:
num_agency_day = pd.read_csv('../src/analysis/num_agency_day.txt', names= ['date', 'type', 'num'])
#print num_agency_day
#num_agency_day = pd.DataFrame(agency_day, columns=['date', 'type', 'num'])
num_agency_day = num_agency_day.sort_values(['date'])

In [21]:
d = datetime.strptime('03/31/2017 01:59:21 PM', "%m/%d/%Y %I:%M:%S %p")
str(d.date())


'2017-03-31'

In [19]:
agency = np.unique(num_agency_day['type'])
y_org = []
for i in agency:
    y_org.append(list(num_agency_day[num_agency_day.type == i]['num']))
y_org = tuple(y_org)
y = np.row_stack(y_org).astype(np.int16)
x = pd.date_range(start='2017-03-01', end='2017-03-31', freq='D')


fig, ax = plt.subplots()
ax.stackplot(x, y)
mpld3.display()

In [64]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# Add original data
x= np.unique(num_agency_day['date']).astype(np.str)

agency = np.unique(num_agency_day['type'])
y_org = []
for i in agency:
    y_org.append(list(num_agency_day[num_agency_day.type == i]['num']))


# Add data to create cumulative stacked values
y_stack = []
for i in range(len(agency)):
    temp = y_org[0]
    for j in range(i):
        temp = [k1 + k2 for k1, k2 in zip(temp, y_org[j+1])]
    y_stack.append(temp)


# Make original values strings and add % for hover text
y_txt = y_org

data = []

for i in range(len(agency)):
    data.append( go.Scatter(
        x=x,
        y=y_stack[i],
        text=y_txt[i],
        mode='lines',
        fill='tonexty',
        name = agency[i]
    ))


fig = go.Figure(data=data)
plot(fig, filename='area.html',image_filename = 'area' ,image='png',auto_open=True)


'file:///Users/xiaodiu/Documents/github/ds1004/figs/area.html'

In [41]:
dir(fig)

['data', 'frames', 'layout']

In [33]:
import os
import folium

print(folium.__version__)

0.3.0


In [37]:
size = 1000
lons = np.random.randint(-180, 180, size=size)
lats = np.random.randint(-90, 90, size=size)

locations = list(zip(lons, lats))
popups = ['{}'.format(loc) for loc in locations]

from folium.plugins import MarkerCluster

m = folium.Map(location=[np.mean(lats), np.mean(lons)],
                  tiles='Cartodb Positron', zoom_start=1)

m.add_child(MarkerCluster(locations=locations, popups=popups))

m.save(os.path.join('1000_MarkerCluster.html'))

m

In [139]:
num_agency_day_box = complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x: ((get_date(x[1]),x[3]), 1)) \
          .foldByKey(0, add) \
          .map(lambda x: ((x[0][0][0], x[0][1]),x[1])) \
          .map(lambda x : "%s,%s,%d" % (x[0][0], x[0][1], x[1])) \
          
num_agency_day_box.saveAsTextFile("num_agency_day_box.out")


In [140]:
num_agency_day_box = pd.read_csv('../Notes/num_agency_day_box.txt', names= ['date', 'agency', 'list'])

num_agency_day_box = num_agency_day_box.sort_values(['date','agency'])


In [147]:
agency = np.unique(num_agency_day_box['agency'])

data = []
for i in agency:
    data.append(go.Box(
        x = list(num_agency_day_box.loc[num_agency_day_box.agency == i,'list']),
        y = list(num_agency_day_box.loc[num_agency_day_box.agency == i,'date']),
        name = i,
        boxmean = False,
        orientation = 'h',
        ))

layout = go.Layout(
    xaxis=dict(
        title='distribution of complaints each year of different type',
        zeroline=False
    ),
    boxmode='group'
)
fig = go.Figure(data=data, layout=layout)

#fig['layout'].update(height=600, width=600, title='Stacked subplots')

plot(fig, filename='box.html')


'file:///Users/xiaodiu/Documents/github/ds1004/figs/box.html'

In [151]:
month_agency_bar = complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x: ((get_date(x[1])[0:2],x[3],x[5]), 1)) \
          .foldByKey(0, add) \
          .map(lambda x: ('_'.join([x[0][0][0],x[0][0][1]]), x[0][1],x[0][2],x[1])) \
          .map(lambda x : "%s,%s,%s,%d" % (x[0], x[1], x[2],x[3]))


month_agency_bar.saveAsTextFile("month_agency_bar.out")

In [152]:
month_agency_bar = pd.read_csv('../Notes/month_agency_bar.txt', names= ['date', 'agency','type','num'])

month_agency_bar = month_agency_bar.sort_values(['date','agency','type'])

In [168]:
month_agency_bar_agency = month_agency_bar.groupby(by = ['date','agency'])['num'].sum()
month_agency_bar_agency = month_agency_bar_agency.reset_index()

agency = np.unique(month_agency_bar_agency['agency'])

data = []
for i in agency:
    data.append(go.Bar(
        x = list(month_agency_bar_agency.loc[month_agency_bar_agency.agency == i,'date']),
        y = list(month_agency_bar_agency.loc[month_agency_bar_agency.agency == i,'num']),
        name = i
        ))

layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='bar_agency.html')

'file:///Users/xiaodiu/Documents/github/ds1004/figs/bar_agency.html'

In [169]:
type__year_pie = complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x: ((get_date(x[1])[0],x[5]), 1)) \
          .foldByKey(0, add) \
          .map(lambda x : "%s,%s,%d" % (x[0][0], x[0][1], x[1]))


type__year_pie.saveAsTextFile("type__year_pie.out")

In [178]:
type__year_pie = pd.read_csv('../Notes/type__year_pie.txt', names= ['year','type','num'])

type__year_pie = type__year_pie.sort_values(['year','type'])

In [184]:
years = np.unique(type__year_pie.year)
num =5
step = (1.0 - 0.04*(num / 2)) / (num / 2 + 1)
ys = [[i*(step + 0.04)  , i*(step + 0.04) + step  ] for i in range((num/2 + 1))]
print ys

[[0.0, 0.3066666666666667], [0.3466666666666667, 0.6533333333333333], [0.6933333333333334, 1.0]]


In [199]:
types = np.unique(type__year_pie.type)
years = np.unique(type__year_pie.year)
num = len(years)
gap = 0.08
step = (1.0 - gap*(num / 2)) / (num / 2 + 1)
ys = [[i*(step + gap)  , i*(step + gap) + step  ] for i in range((num/2 + 1))]
xs = [[0, .48],[.52, 1]]
data = []
annotations = []
for i,j in enumerate(years):
    data.append(go.Pie(
        values = list(type__year_pie.loc[type__year_pie.year == j, 'num']),
        labels = list(type__year_pie.loc[type__year_pie.year == j, 'type']),
        domain = dict(
                x = xs[i % 2], 
                y = ys[i / 2]),
        name = j,
        textinfo ='none' , 
        hole = .4
        ))
    
    annotations.append(dict(
        font = dict(
            size = 20
            ),
        showarrow = False,
        text = j,
        x = (sum(xs[i % 2]) - 0.08 )/ 2.0,
        y = sum(ys[i / 2]) / 2.0)
        )

layout = go.Layout(
    title = 'Complaint type distribution every year',
    annotations = annotations
)

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='type__year_pie.html')

'file:///Users/xiaodiu/Documents/github/ds1004/figs/type__year_pie.html'